In [ ]:
!pip install openai

In [ ]:
import os
openrouter_key=os.getenv("openrouter_key")

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openrouter_key,
)
'''
completion = client.chat.completions.create(
  extra_body={},
  model="tngtech/deepseek-r1t2-chimera:free",
  messages=[
    {
      "role": "user",
      "content": "Hello, how are you?"
    }
  ]
)
print(completion.choices[0].message.content)
''';

In [ ]:

import json

from IPython.display import display, HTML
import markdown

class Tools:
    def __init__(self):
        self.tools = {}
        self.functions = {}

    def add_tool(self, function, description):
        self.tools[function.__name__] = description
        self.functions[function.__name__] = function

    def get_tools(self):
        return list(self.tools.values())

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        f = self.functions[function_name]
        result = f(**arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }


def shorten(text, max_length=50):
    if len(text) <= max_length:
        return text

    return text[:max_length - 3] + "..."


class ChatInterface:
    def input(self):
        question = input("You:")
        return question

    def display(self, message):
        print(message)

    def display_function_call(self, entry, result):
        call_html = f"""
            <details>
            <summary>Function call: <tt>{entry.name}({shorten(entry.arguments)})</tt></summary>
            <div>
                <b>Call</b>
                <pre>{entry}</pre>
            </div>
            <div>
                <b>Output</b>
                <pre>{result['output']}</pre>
            </div>

            </details>
        """
        display(HTML(call_html))

    def display_response(self, entry):
        response_html = markdown.markdown(entry.content[0].text)
        html = f"""
            <div>
                <div><b>Assistant:</b></div>
                <div>{response_html}</div>
            </div>
        """
        display(HTML(html))



class ChatAssistant:
    def __init__(self, tools, developer_prompt, chat_interface, client):
        self.tools = tools
        self.developer_prompt = developer_prompt
        self.chat_interface = chat_interface
        self.client = client

    def gpt(self, chat_messages):
        return self.client.responses.create(
            model="openrouter/cypher-alpha:free",
            input=chat_messages,
            tools=self.tools.get_tools(),
        )


    def run(self):
        chat_messages = [
            {"role": "developer", "content": self.developer_prompt},
        ]

        # Chat loop
        while True:
            question = self.chat_interface.input()
            if question.strip().lower() == 'stop':
                self.chat_interface.display("Chat ended.")
                break

            message = {"role": "user", "content": question}
            chat_messages.append(message)

            while True:  # inner request loop
                response = self.gpt(chat_messages)

                has_messages = False

                print(response)

                display(HTML(response))

                for entry in response.output:
                    chat_messages.append(entry)

                    if entry.type == "function_call":
                        result = self.tools.function_call(entry)
                        chat_messages.append(result)
                        self.chat_interface.display_function_call(entry, result)

                    elif entry.type == "message":
                        self.chat_interface.display_response(entry)
                        has_messages = True

                if has_messages:
                    break


In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "function that returns today's temperature in a particular city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {     # <--
                "type": "string",
                "description": "designated city"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [ ]:
tools = Tools()
tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

developer_prompt = """
You're a weather forecasting assistant.
You're given a question from a citizen about today's temperature in a city and you role is to answer it.

Use the weather forecasting tool if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = ChatInterface()

chat = ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()

In [3]:

def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'


get_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "function that registers today's temperature for a particular city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {     # <--
                "type": "string",
                "description": "designated city"
            },
            "temp": {     # <--
                "type": "float",
                "description": "temperature"
            }
        },
        "required": ["city","temp"],
        "additionalProperties": False
    }
}

In [4]:
!pip install fastmcp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 1.1 MB/s eta 0:00:00-:--:--
  DEPRECATION: Building 'pyperclip' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pyperclip'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11103 sha256=2b050f25cecc3561ced8299c41b74d61572338860b7926794b6128f0f2c5b7f7
  Stored in directory: /root/.cache/pip/wheels/e0/e8/fc/8ab8aa326e33bc066ccd5f3ca9646eab4299881af933f94f09
Successfully built pyperclip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18/18 [fastmcp]7/18 [fastmcp]]ydantic]


In [5]:
!pip list | grep fastmcp

fastmcp                   2.10.4


In [ ]:
#[07/11/25 19:38:53] INFO     Starting MCP server 'Demo 🚀' with transport 'stdio' 

In [7]:
{"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"get_weather","description":"Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.","inputSchema":{"properties":{"city":{"title":"City","type":"string"}},"required":["city"],"type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"number"}},"required":["result"],"title":"_WrappedResult","type":"object","x-fastmcp-wrap-result":True}},{"name":"set_weather","description":"Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.","inputSchema":{"properties":{"city":{"title":"City","type":"string"},"temp":{"title":"Temp","type":"number"}},"required":["city","temp"],"type":"object"}}]}}

{'jsonrpc': '2.0',
 'id': 2,
 'result': {'tools': [{'name': 'get_weather',
    'description': 'Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.',
    'inputSchema': {'properties': {'city': {'title': 'City',
       'type': 'string'}},
     'required': ['city'],
     'type': 'object'},
    'outputSchema': {'properties': {'result': {'title': 'Result',
       'type': 'number'}},
     'required': ['result'],
     'title': '_WrappedResult',
     'type': 'object',
     'x-fastmcp-wrap-result': True}},
   {'name': 'set_weather',
    'description': "Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.",
    'inputSchema': {'pro

In [ ]:
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city":"berlin"}}}

In [9]:
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":False}}

{'jsonrpc': '2.0',
 'id': 3,
 'result': {'content': [{'type': 'text', 'text': '20.0'}],
  'structuredContent': {'result': 20.0},
  'isError': False}}


Available tools: [Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'}, outputSchema=None, annotations=None, meta=None)]
